In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

### Functions

In [2]:
def high_pass_filter(ht, wd):
    eta = np.cos(np.pi * np.linspace(-0.5, 0.5, num=ht)).reshape(1, ht)
    neta = np.cos(np.pi * np.linspace(-0.5, 0.5, num=wd)).reshape(1, wd)

    X = np.dot(eta.T, neta)

    H = (1.0 - X) * (2.0 - X)

    return H

In [110]:
def log_polar(img):
    img_lp = None
    
    Y, X = img.shape
    
    # find center
    Xc = (1 + X) / 2
    Yc = (1 + Y) / 2
    
    # Calculate log base and angle increments
    base = np.exp(np.log(Xc - 1) / (Xc - 1))
    dtheta = np.pi/Y
    
    # Build x-y coordinates of log-polar points
    Xin = np.zeros([Y,X], np.float32)
    Yin = np.zeros([Y,X], np.float32)

    for y in range(Y):
        theta = y * dtheta
        for x in range(X):
            r = base**(x/2) - 1
            Xin[y, x] = r * np.cos(theta) + Xc
            Yin[y, x] = r * np.sin(theta) + Yc

    # remap
    img_lp = cv2.remap(img, Xin, Yin, cv2.INTER_CUBIC)
    
    img_lp = np.nan_to_num(img_lp)
    
    return img_lp, base

## Loading image and generate new image

In [122]:
img_r = cv2.imread('lena.png', 0)
rows, cols = img_r.shape

#########################################################
# create a transfered image
# translation testing
tx = 105.
ty = -115.
theta = 45
scale = 1.2

M_T = np.float32([[1, 0, tx],
                  [0, 1, ty]])

img_s = cv2.warpAffine(img_r, M_T, (cols, rows))

M_RS = cv2.getRotationMatrix2D((cols/2, rows/2), theta, scale)
img_s = cv2.warpAffine(img_s, M_RS, (cols, rows))

# plt.figure()
# plt.imshow(img_r, cmap='gray')
# plt.title('Input Image 1'), plt.xticks([]), plt.yticks([])
# plt.figure()
# plt.imshow(img_s, cmap='gray')
# plt.title('Input Image 2'), plt.xticks([]), plt.yticks([])
# plt.show()

### Scaling Estimation

In [123]:
# fourier transfer
f_r = np.fft.fft2(img_r)
f_r_shift = np.fft.fftshift(f_r)
f_r_mag = np.abs(f_r_shift)

f_s = np.fft.fft2(img_s)
f_s_shift = np.fft.fftshift(f_s)
f_s_mag = np.abs(f_s_shift)

# plt.subplot(221), plt.imshow(np.log(f_r_mag), cmap='gray')
# plt.title('Spectrum Image R'), plt.xticks([]), plt.yticks([])
# plt.subplot(222), plt.imshow(np.log(f_s_mag), cmap='gray')
# plt.title('Spectrum Image S'), plt.xticks([]), plt.yticks([])

# high pass filter
H = high_pass_filter(rows, cols)
f_r_hp = H * f_r_mag
f_s_hp = H * f_s_mag

# plt.subplot(223), plt.imshow(f_r_hp, cmap='gray')
# plt.title('HP Spectrum Image R'), plt.xticks([]), plt.yticks([])
# plt.subplot(224), plt.imshow(f_s_hp, cmap='gray')
# plt.title('HP Spectrum Image S'), plt.xticks([]), plt.yticks([])

# log-polar and cross
lp_r, _ = log_polar(f_r_hp)
cp_r = np.fft.fft2(lp_r)

lp_s, base = log_polar(f_s_hp)
cp_s = np.fft.fft2(lp_s)

# phase shift and cross power spectrum
phase_shift = np.exp(1j * (np.angle(cp_r) - np.angle(cp_s)))
cps = np.real(np.fft.ifft2(phase_shift))

y_max, x_max = np.unravel_index(cps.argmax(), cps.shape)

theta = 180 * (y_max) / rows
if theta > 90:
    theta -= 180
    
scale = 0
if x_max > cols / 2:
    scale = x_max - cols
else:
    scale = x_max 

scale = base**(scale/2)

print(theta, scale)

45.0 1.2025117415216577


In [1]:
import time

In [2]:
start_time = time.time()
for y in range(512):
    for x in range(512):
        c = x + y
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.02872633934020996 seconds ---
